### Imports, Simulation Setup

In [1]:
import numpy as np 
import pandas as pd
from pulsee import simulation as sim, quantum_computing as qc, operators as op 
from pulsee.many_body import tensor_product

In [2]:
Iz = np.array([[1/2, 0], [0,-1/2]])
Iz = op.Operator(Iz)

In [3]:
SIM_NAME = "/test/11uncorrelated_mixed_no_interactions"

### Quantum System Setup
Spin-$\frac{1}{2}$ system with $\gamma/2\pi = 4.00$ MHz/T (see Candoli thesis §4.1) and $B_0 = 10$ T:

In [4]:
CORRELATED = True
MIXED = True
QUANTUM_NUMBERS = [1.5, 0.5] 
GAMMA_2PIS = [4.00, 0] 
B_0 = 10
# one control qubit (cnotnot) or two control qubits (toffoli)
CNOT_GATE = qc.toffoli 

zeem_par = {'field magnitude' : B_0,
                'theta_z' : 0,
                'phi_z' : 0}

### Pulse
Set up single mode pulse of form 
$$\mathbf{B}_1(t) = 2 B_1 \cos(2\pi \nu + \delta)\cdot \hat{\mathbf{n}}(\theta, \phi)$$
where $\theta$ and $\phi$ are the polar and azimuthal angles respectively. $B_1$ in T, $\nu$ in MHz, $\delta$, $\phi$, and $\theta$ in radians. TODO: $\nu$ depends on $e^2 q Q $ and $\eta$? What is $\eta$? Why?

For simplest case apply $\pi/2$ pulse with $B_1 = B_0 / 100 $, $ \nu = (\gamma/2\pi) \cdot B_0 $, $\delta = \phi =0$, $\theta=\pi/2$.

In [5]:
B_1 = B_0 / 100
FREQ = GAMMA_2PIS[0] * B_0
PHASE = 0
POLAR_B = np.pi / 2 
AZIMUTHAL_B = 0 
PULSE_TIME = 1 / (4 * FREQ)
mode = pd.DataFrame([(FREQ, 2 * B_1, PHASE, POLAR_B, AZIMUTHAL_B)],
					columns=['frequency', 'amplitude', 'phase', 'theta_p', 'phi_p'])

### Experiment Settings

In [6]:
ACQUISITION_TIME = 1000 # in microseconds
T2 = 100

### Hamiltonians
Here we add the desired Hamiltonian interaction terms. Add these to the following dictionary `args` with key as it appears in `nuclear_system_setup` function. The value should be the parameters of the associated Hamiltonian as it appears in `nuclear_system_setup`.

In [7]:
h_dict = {
	'h_tensor_inter': np.array([[2, 0, 0], [0, 1, 0], [0, 0, 1]])
}

### Density Matrix Setup 
If `CORRELATED` start with qubit $\ket{111}$ so that in the case of 
Toffoli (CCNOT) the second control qubit leads to negation of last qubit. 
Before applying CCNOT or CNOTNOT apply Hadamard to first qubit to create 
superposition. 

In [18]:
args = h_dict.copy()
spin_par = []
for qn, gam in zip(QUANTUM_NUMBERS, GAMMA_2PIS):
    spin_par.append({'quantum number': qn, 'gamma/2pi': gam})

args['spin_par'] = spin_par 
args['zeem_par'] = zeem_par
spin, h_unperturbed, _ = sim.nuclear_system_setup(**args)

qs = qc.CompositeQubitSpace(3)

# Construct the tensor product of the Hadamard gate and 2 identity operators
had_id = qc.gate_tensor_product(qc.hadamard, qc.gate_tensor_pow(qc.identity, 2))

corr_operator = CNOT_GATE * had_id


if CORRELATED and MIXED: 
    state = op.canonical_density_matrix(op.Operator(np.sum(h_unperturbed, axis=0)), 1e-4).matrix
    print(state.shape)
    print(had_id.matrix.shape)
    corr = had_id.matrix @ state @ qc.adjoint(had_id.matrix)

    initial_dm_matrix = corr
    
elif MIXED: 
    initial_dm_matrix = op.canonical_density_matrix(op.Operator(np.sum(h_unperturbed), axis=0), 1e-4).matrix
    
elif CORRELATED: 
    # Use state ket |111⟩
    q = qs.basis_ket_from_indices([0, 0, 0])

    # Create entanglement 
    corr_state = had_id(q)
    initial_dm = corr_state.density_matrix
    initial_dm_matrix = initial_dm.matrix

initial_dm = op.Operator(initial_dm_matrix)

print('H_unperturbed:')
print(np.round(np.sum(h_unperturbed, axis=0), 3))
print('Initial density matrix:')
print(np.round(initial_dm.matrix, 3))

(8, 8)
(8, 8)
H_unperturbed:
[[-59.25 +0.j   0.   +0.j   0.   +0.j   0.433+0.j   0.   +0.j   0.   +0.j
    0.   +0.j   0.   +0.j]
 [  0.   +0.j -60.75 +0.j   1.299+0.j   0.   +0.j   0.   +0.j   0.   +0.j
    0.   +0.j   0.   +0.j]
 [  0.   +0.j   1.299+0.j -19.75 +0.j   0.   +0.j   0.   +0.j   0.5  +0.j
    0.   +0.j   0.   +0.j]
 [  0.433+0.j   0.   +0.j   0.   +0.j -20.25 +0.j   1.5  +0.j   0.   +0.j
    0.   +0.j   0.   +0.j]
 [  0.   +0.j   0.   +0.j   0.   +0.j   1.5  +0.j  19.75 +0.j   0.   +0.j
    0.   +0.j   0.433+0.j]
 [  0.   +0.j   0.   +0.j   0.5  +0.j   0.   +0.j   0.   +0.j  20.25 +0.j
    1.299+0.j   0.   +0.j]
 [  0.   +0.j   0.   +0.j   0.   +0.j   0.   +0.j   0.   +0.j   1.299+0.j
   59.25 +0.j   0.   +0.j]
 [  0.   +0.j   0.   +0.j   0.   +0.j   0.   +0.j   0.433+0.j   0.   +0.j
    0.   +0.j  60.75 +0.j]]
Initial density matrix:
[[ 0.162+0.j  0.   +0.j  0.   +0.j -0.002+0.j  0.162+0.j  0.   +0.j
   0.   +0.j -0.002+0.j]
 [ 0.   +0.j  0.338+0.j -0.011+0.j  0.   +0.j

### Simulation

In [9]:
evolved_dm = sim.evolve(spin, h_unperturbed, initial_dm, 
						mode=mode, pulse_time=PULSE_TIME)

t, fid = sim.FID_signal(spin, h_unperturbed, evolved_dm, 
						acquisition_time=ACQUISITION_TIME, T2=T2)
						
f, ft = sim.fourier_transform_signal(t, fid, -1, 1)

KeyboardInterrupt: 

Write simulation parameters and save plots to a directory. 